In [1]:
# import os 
# from tqdm import tqdm
# import shutil 
# import random

# image_paths_and_classes = []

# for file_name in os.listdir("./train_generated/no_watermark/"):
#     image_paths_and_classes.append(("./train_generated/no_watermark/"+file_name,"no_watermark", file_name))

# for file_name in os.listdir("./train_generated/watermark/"):
#     image_paths_and_classes.append(("./train_generated/watermark/"+file_name,"watermark", file_name))

# random.shuffle(image_paths_and_classes)
# testing = image_paths_and_classes[int(len(image_paths_and_classes)*0.8):]   #0.2

# for obj in testing:
#     shutil.move(obj[0],f"./train_generated/test/{obj[1]}/"+obj[2])

In [2]:
# import os 
# from tqdm import tqdm
# from PIL import Image
# deleted=0
# paths= ["./train_generated/watermark/","./train_generated/no_watermark/"]
# for p in paths:
#     for file_name in tqdm(os.listdir(p)):
#         try:
#             img = Image.open(p+file_name)
#             img.load()
#         except:
#             os.remove(p+file_name)
#             deleted+=1
# print(deleted)

In [3]:
# import os 
# from tqdm import tqdm
# from PIL import Image
# deleted=0
# paths= ["./train_generated/watermark/","./train_generated/no_watermark/","./watermark/","./no_watermark/"]
# for p in paths:
#     for file_name in tqdm(os.listdir(p)):
#         if "_aug_" in file_name:
#             os.remove(p+file_name)
#             deleted+=1
# print(deleted)

In [4]:
# import os 
# from tqdm import tqdm
# from PIL import Image
# import numpy as np
# deleted=0
# paths= ["./train_generated/watermark/","./train_generated/no_watermark/"]
# for p in paths:
#     for file_name in tqdm(os.listdir(p)):
#         if "_aug_" in file_name:
#             # os.remove(p+file_name)
#             # deleted+=1
#             pass
#         else:
#             img = Image.open(p+file_name)
#             if img.size[0]!=512 or img.size[1]!=512:
#                 img=img.resize((512,512),Image.Resampling(np.random.randint(1,6)))    #[0,5)
#                 img.save(p+file_name,quality=90)
# # print(deleted)

In [5]:
# import os
# import albumentations as A
# import numpy as np
# all_transforms = [
#   A.Compose([
#     A.augmentations.geometric.rotate.SafeRotate(limit=30,value=[0,0,0], border_mode=0, p=0.10),
#     A.core.composition.OneOf([A.Lambda(p=1), A.VerticalFlip(p=1), A.HorizontalFlip(p=1),A.RandomRotate90(p=1)],p=1),
#     A.core.composition.OneOf([A.Lambda(p=1), A.augmentations.ToGray(p=1), A.augmentations.RGBShift(p=1),
#     A.augmentations.ChannelShuffle(p=1),A.augmentations.ColorJitter(p=1),A.InvertImg(p=1)],p=1),
#     A.core.composition.OneOf([A.Lambda(p=1),A.augmentations.MotionBlur(p=1), A.augmentations.AdvancedBlur(p=1), A.augmentations.GaussianBlur(p=1)],p=1),
#     A.ImageCompression(p=0.20, quality_lower=35),
# ])
# ]

# from tqdm import tqdm
# from PIL import Image
# paths= ["./train_generated/watermark/","./train_generated/no_watermark/","./no_watermark/","./watermark/"]
# for p in paths:
#     for file_name in tqdm(os.listdir(p)):
#           img = Image.open(p+file_name)
#           if img.mode != 'RGB':
#             img = img.convert('RGB')
#           # img=img.resize((512,512),Image.Resampling(np.random.randint(6)))    #[0,5)
#           img = np.array(img)
#           for i in range(1):
#             new_img = all_transforms[0](image=img)["image"]
#             # print(new_img.shape)
#             new_img =  Image.fromarray(new_img,"RGB")

#             new_file_name = f'{file_name[:file_name.index(".")]}_aug_{str(i)}_{file_name[file_name.index("."):]}'
#             new_img.save(p+new_file_name,quality=90)


In [1]:
import torch
import timm

torch.set_float32_matmul_precision('high')
device="cuda"


/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def create_default_model():
    model = timm.create_model('tf_efficientnetv2_b0', pretrained=True,drop_path_rate=0.2,drop_rate=0.2)
    model.classifier=torch.nn.Linear(1280,1)

    # for param in model.parameters():
    #     param.requires_grad = False
    
    # for blk in [model.classifier, model.global_pool,model.conv_head,model.blocks[-1],model.blocks[-2],model.blocks[-3]]:
    #     for param in blk.parameters():
    #         param.requires_grad = True

    # for module in model.modules():
    #     if isinstance(module, torch.nn.BatchNorm2d):
    #         if hasattr(module, 'weight'):
    #             module.weight.requires_grad_(False)
    #         if hasattr(module, 'bias'):
    #             module.bias.requires_grad_(False)
    #         module.eval()
    model = model.to(device)
    # model=torch.compile(model)
    return model

In [8]:
model = create_default_model()

In [2]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os 
import random

def read_img_file(f):
    img = Image.open(f)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img


_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

class CustomDataset(Dataset):
    def __init__(self, image_paths_labels,):
        self.image_paths_labels = image_paths_labels

    def __len__(self):
        return len(self.image_paths_labels)

    def __getitem__(self, idx):
        img_path = self.image_paths_labels[idx][0]
        label = self.image_paths_labels[idx][1]
        try:
            img = read_img_file(img_path)
            img = np.array(img)
            # img = all_transforms[0](image=img)["image"]
            img = _transform(img)
            return (img, label)
        except Exception as e:
            print(e)
            print(f"error reading {img_path}")


image_paths_and_classes = []


for file_name in os.listdir("./no_watermark/"):
    image_paths_and_classes.append(("./no_watermark/"+file_name, 0.0))

for file_name in os.listdir("./watermark/"):
    image_paths_and_classes.append(("./watermark/"+file_name, 1.0))

# random.shuffle(image_paths_and_classes)
# training = image_paths_and_classes[:int(len(image_paths_and_classes)*0.8)]
# testing = image_paths_and_classes[int(len(image_paths_and_classes)*0.8):]   #0.2

# image_paths_and_classes=[]
for file_name in os.listdir("./train_generated/no_watermark/"):
    image_paths_and_classes.append(("./train_generated/no_watermark/"+file_name, 0.0))

for file_name in os.listdir("./train_generated/watermark/"):
    image_paths_and_classes.append(("./train_generated/watermark/"+file_name, 1.0))

random.shuffle(image_paths_and_classes)
training = image_paths_and_classes

testing = []
for file_name in os.listdir("./test/no_watermark/"):
    testing.append(("./test/no_watermark/"+file_name, 0.0))

for file_name in os.listdir("./test/watermark/"):
    testing.append(("./test/watermark/"+file_name, 1.0))

# training.extend(image_paths_and_classes[:int(len(image_paths_and_classes)*0.8)])
# random.shuffle(training)

# testing2 = image_paths_and_classes[int(len(image_paths_and_classes)*0.8):]   #0.2
    

In [10]:
BATCH_SIZE=64
train_dataset = CustomDataset(training)
test_dataset = CustomDataset(testing)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,shuffle=True,num_workers=2, prefetch_factor=4,pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=True,num_workers=2, prefetch_factor=4,pin_memory=True)

In [12]:
from tqdm import tqdm
from timeit import default_timer as timer
from statistics import mean
from torch.cuda.amp import GradScaler
from torch import autocast
import optuna
import wandb
import numpy as np
from sklearn.metrics import accuracy_score

criterion = torch.nn.BCEWithLogitsLoss()

def train(lr,trial=None,EPOCHS=15):
    wandb.init(project="effnet_b0_watermark", entity="qwertyforce",reinit=True)

    wandb.config.update({
        "learning_rate": lr,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        })
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr) 
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scaler = GradScaler()

    loss_train=[]
    loss_test=[]
    acc_test=[]
    for epoch in tqdm(range(EPOCHS)):
        train_one_epoch(loss_train,optimizer,criterion,scaler)
        test(loss_test,acc_test,criterion)
        if wandb:
            wandb.log({"loss_train": loss_train[-1],"epoch":epoch})
            wandb.log({"loss_test": loss_test[-1],"epoch":epoch})
            wandb.log({"acc_test": acc_test[-1],"epoch":epoch})
        # if trial:
        #     trial.report(mean(loss_test), epoch)
        #     if trial.should_prune():
        #         raise optuna.exceptions.TrialPruned()
    return mean(loss_test)


def train_one_epoch(loss_train,optimizer,criterion,scaler):
    model.train()
    temp_train_loss=[]
    start = timer()
    for batch_idx, (data, labels) in enumerate(train_dataloader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model.forward(data)
            loss = criterion(outputs,labels.unsqueeze(1))
        
        # outputs = model.forward(data)
        # loss = criterion(outputs,labels.unsqueeze(1))
        # loss.backward()
        # optimizer.step()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        temp_train_loss.append(loss.item())
        # if batch_idx % 10==0:
    end = timer()
    print(f"epoch_training_time: {end - start}") 
    loss_train.append(mean(temp_train_loss))
    print(f"Train: {mean(temp_train_loss)}")

            

def test(loss_test,acc_test, criterion):
    model.eval()
    temp_loss=[]
    predictions, true_labels = [], []
    for batch_idx, (data, labels) in enumerate(test_dataloader):
        with torch.no_grad():
          true_labels.extend(labels)
          data, labels = data.to(device), labels.to(device)
          outputs = model.forward(data)
          loss = criterion(outputs,labels.unsqueeze(1))
          temp_loss.append(loss.item())

          outputs = torch.sigmoid(outputs).cpu().numpy()
          outputs = np.round(outputs)
          predictions.extend(outputs)


    loss_test.append(mean(temp_loss))
    acc_test.append(accuracy_score(true_labels,predictions))
    print(f"Test: {mean(temp_loss)}")
    # return mean(temp_loss)



In [ ]:
train(0.001,EPOCHS=10)

In [14]:
assert 1==2

AssertionError: 

In [ ]:
# import optuna
# def objective(trial):
#     global model
#     model = create_default_model()
#     lr = trial.suggest_loguniform('learning_rate', 1e-6, 1e-2)
#     mean_loss = train(lr)
#     return mean_loss

# study = optuna.create_study(direction='minimize',pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=10)

In [ ]:
# from optuna.visualization import plot_contour
# from optuna.visualization import plot_edf
# from optuna.visualization import plot_intermediate_values
# from optuna.visualization import plot_optimization_history
# from optuna.visualization import plot_parallel_coordinate
# from optuna.visualization import plot_param_importances
# from optuna.visualization import plot_slice

In [ ]:
# plot_slice(study)


In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(loss_train)
# plt.plot(loss_test)

In [ ]:

# test_dataset = CustomDataset(testing2)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,shuffle=True,num_workers=2, prefetch_factor=8,pin_memory=True)

model.eval()
predictions, true_labels= [], []
raw_values=[]
import numpy as np
with torch.no_grad():
    for batch_idx, (data, labels) in enumerate(test_dataloader):
        data, labels = data.to(device), labels.to(device)
        outputs = model.forward(data)
        outputs = torch.sigmoid(outputs).cpu().numpy()
        raw_values.extend(outputs.reshape(-1))
        outputs = np.round(outputs)
        predictions.extend(outputs)
        true_labels.extend(labels.cpu().numpy())
        # loss = criterion(outputs,labels.unsqueeze(1))  

In [17]:
from sklearn.metrics import classification_report



In [18]:
print(classification_report(true_labels,predictions))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      2592
         1.0       0.98      0.96      0.97      1959

    accuracy                           0.97      4551
   macro avg       0.97      0.97      0.97      4551
weighted avg       0.97      0.97      0.97      4551



In [ ]:
assert 1==2

AssertionError: 

In [8]:
p = "/media/qwertyforce/26fbdf65-ba8c-46bc-bbd9-bc503969e854/scenery_cx/scenery/public/images/"
model.eval()
model=model.to("cuda")
# p="./test/no_watermark/"
res=[]
all_outputs=[]
for file_name in tqdm(os.listdir(p)):
    img = read_img_file(p+file_name)
    img=img.resize((512,512),Image.Resampling.NEAREST)
    img = _transform(img).cuda()
    img.unsqueeze_(0)
    img = img.to(device)
    with torch.no_grad():
        outputs = model.forward(img)
        outputs = torch.sigmoid(outputs).cpu().numpy()
        all_outputs.append(outputs[0][0])
    if np.round(outputs[0][0]) == 1:
        res.append((file_name,outputs[0][0]))


  0%|          | 381/165036 [01:20<8:06:07,  5.65it/s] /usr/local/lib/python3.9/dist-packages/PIL/Image.py:3074: DecompressionBombWarning: Image size (129957065 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  1%|          | 976/165036 [03:11<7:31:58,  6.05it/s] 

In [ ]:
len(res)

236

In [ ]:
# torch.save(model, './model.pt')

In [ ]:
# import onnxruntime as onnx
# model.eval()
# model=model.to("cpu")
# x = torch.randn(1, 3, 512, 512, requires_grad=False).cpu()
# # torch_out = model(x)
# torch.onnx.export(model,                     # model being run
#                   x,                            # model input (or a tuple for multiple inputs)
#                   "model.onnx",              # where to save the model (can be a file or file-like object)
#                   export_params=True,           # store the trained parameter weights inside the model file
#                   opset_version=12,             # the ONNX version to export the model to
#                   do_constant_folding=True,     # whether to execute constant folding for optimization
#                   input_names = ['input'],      # the model's input names
#                   output_names = ['output'],
#                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
#                                 'output' : {0 : 'batch_size'}})    # the model's output names

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
# import torch
# import onnxruntime
# import numpy as np

In [ ]:
# model = torch.load("model.pt")
# model=model.to("cpu")
# model.eval()

# inp_arr = torch.randn(32,3,512,512)

# with torch.no_grad():
#     out_1 = model(inp_arr)


# sess_options = onnxruntime.SessionOptions()
# sess_options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
# sess_options.enable_cpu_mem_arena=False
# session = onnxruntime.InferenceSession("./model.onnx", sess_options, providers=['CPUExecutionProvider'])
# out_2 = session.run([], {'input':input_arr.numpy()})[0]